# Aula 8 - Solução dos exercícios - Parte 3 - Inpars

Leandro Carísio Fernandes

<br>

In [1]:
#É necessário rodar essa célula primeiro antes de rodar o resto do caderno
!pip install -qU huggingface_hub datasets

from huggingface_hub import login
login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.3 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Thu May  4 13:01:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
# Dados TREC-COVID
url_trec_covid = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

nome_tokenizer = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
nome_modelo = 'cross-encoder/ms-marco-MiniLM-L-6-v2' # Podemos abrir aqui o modelo lá da aula 2 também, pra testar

# Dados do modelo pra treinar
treinar_modelo = True
url_salvar_modelos_intermediarios = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula8-inpars/modelos/1neg_1pos_teste_apaga_token_type_id_no_treino/'

# Modelo eval
modelo_eval = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula8-inpars/modelos/1neg_1pos_teste_apaga_token_type_id_no_treino/validacao/'

max_seq_length = 512
batch_size = 32 # T4: 16, V100: 32, A100: 64
lr = 5e-5
perc_treino = 0.9
epochs = 10

max_ex_negativo = 1
classe_pos = '1'
classe_neg = '0'

In [4]:
%%time
# Já monta o drive, pois vamos usar o índice invertido da Aula 1 para usar o BM25 implementado também na aula 1
# Além disso, é necessário para salvar/recuperar o modelo tunado
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
CPU times: user 920 ms, sys: 199 ms, total: 1.12 s
Wall time: 22.7 s


In [ ]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
# %%shell
# pip install pyserini -q
# pip install faiss-cpu  -q
# apt-get install maven -qq
# git clone --recurse-submodules https://github.com/castorini/pyserini.git
# cd pyserini
# cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
# cd tools/eval/ndeval && make && cd ../../..

In [5]:
%%time
!pip install transformers datasets -q
!pip install sentence-transformers -q
!pip install pyserini -q
!pip install faiss-gpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00
CPU times: user 1.56 s, sys: 221 ms, to

## Carrega a coleção TREC-COVID

In [6]:
%%time
from pathlib import Path
import json
    
if not Path('./collections/trec-covid.zip').is_file():
  !wget {url_trec_covid} -P collections # type: ignore
  !unzip -o collections/trec-covid.zip -d ./collections # type: ignore

# Converte o qrels que veio no trec-covid.zip pra o formato esperado:
with open('./collections/trec-covid/qrels/test.tsv', 'r') as fin:
  data = fin.read().splitlines(True)
with open('./collections/trec-covid/qrels/test_corrigido.tsv', 'w') as fout:
  for linha in data[1:]:
    campos = linha.split()
    fout.write(f'{campos[0]}\t0\t{campos[1]}\t{campos[2]}\n')
    
def carrega_corpus_trec_covid():
  retorno = {}
  with open('./collections/trec-covid/corpus.jsonl') as corpus:
    for i, line in enumerate(corpus):
      doc = json.loads(line)
      retorno[doc['_id']] = f"{doc['title']} {doc['text']}"
      if (i % 10000 == 0):
        print(f'Processado {i} documentos')
    return retorno

def carrega_queries_trec_covid():
  retorno = []
  with open('./collections/trec-covid/queries.jsonl') as queries:
    for line in queries:
      query = json.loads(line)
      # Faz apenas uma pequena tradução de _id para id e text para texto
      retorno.append({'id': query['_id'], 'texto': query['text']})
  return retorno

queries_trec_covid = carrega_queries_trec_covid()
corpus_trec_covid = carrega_corpus_trec_covid()

--2023-05-04 13:07:06--  https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73876720 (70M) [application/zip]
Saving to: ‘collections/trec-covid.zip’

trec-covid.zip      100%[===================>]  70.45M   174MB/s    in 0.4s    

2023-05-04 13:07:06 (174 MB/s) - ‘collections/trec-covid.zip’ saved [73876720/73876720]

Archive:  collections/trec-covid.zip
   creating: ./collections/trec-covid/
   creating: ./collections/trec-covid/qrels/
  inflating: ./collections/trec-covid/qrels/test.tsv  
  inflating: ./collections/trec-covid/corpus.jsonl  
  inflating: ./collections/trec-covid/queries.jsonl  
Processado 0 documentos
Processado 10000 documentos
Processado 20000 documentos

## Cria datasets e dataloaders de treino e validação

In [7]:
# Recupera o dataset que será usado
import datasets
ds = datasets.load_dataset('unicamp-dl/trec-covid-experiment')

Generating example split: 0 examples [00:00, ? examples/s]

Generating example2 split: 0 examples [00:00, ? examples/s]

Generating eduseiti_100_queries_expansion_20230501_01 split: 0 examples [00:00, ? examples/s]

Generating leandro_carisio_01 split: 0 examples [00:00, ? examples/s]

Generating thales_1k_generated_queries_20230429 split: 0 examples [00:00, ? examples/s]

...

Generating ricardo_primi_1k split: 0 examples [00:00, ? examples/s]

Generating thiago_vieira_1k_queries split: 0 examples [00:00, ? examples/s]

Generating eduseiti_1000_queries_expansion_20230502_02 split: 0 examples [00:00, ? examples/s]

Dataset trec-covid-experiment downloaded and prepared to /root/.cache/huggingface/datasets/unicamp-dl___trec-covid-experiment/default/0.0.0/5dba10045db0dc33df357c18a85d3803a49c453a7eeb9daac832882e92fc5d79. Subsequent calls will reuse this data.


  0%|          | 0/21 [00:00<?, ?it/s]

Pra facilitar, coloca o dataset em um dataframe Pandas e deixa ele no formato longo. Faz os seguintes passos:

1. Remove dados de exemplo
2. Deixa no máximo 5 exemplos negativos por query
3. Renomeia positive_doc_id e negative_doc_ids para, respectivamente, 1 e 0
4. A coluna negatieve_doc_ids é uma lista. Cria uma entrada pra cada elemento da lista replicando todas as outras colunas
5. Faz o melt nas colunas 0 e 1 pra deixar o dataframe no formato longo.
6. Altera a coluna variable para label (pois representa 0 e 1) e value para passage. Nesse ponto aqui a passage é, ainda, o doc_id
7. No passo de expandir a lista de docs, negativos, se não tiver elemento na lista a operação explode vai colocar nan. Então filtra os casos de passage com nan
8. Agora converte efetivamente o doc_id na passage

Nesse ponto o dataframe tem 4 colunas: query, passage, origin, label.

Separa o dataframe em dois, um de treino e um de teste. Antes disso, embaralha as linhas do dataset.

In [8]:
import pandas as pd
# Cria um dataframe a partir do dataset HF
df = pd.concat((v.to_pandas().assign(origin=k) for k,v in ds.items()), ignore_index=True)
# Remove os registros de example e example2
df = df[(df.origin != 'example') & (df.origin != 'example2')]
# Deixa no máximo 5 exemplos negativos 
df.negative_doc_ids = df.negative_doc_ids.map(lambda vec: vec[0:max_ex_negativo])
# Altera o label de positive_doc_id e negative_doc_ids para 1 e 0, pois esses serão os labels usados lá no treinamento
df.rename(columns={'positive_doc_id': classe_pos, 'negative_doc_ids': classe_neg}, inplace=True)
# Expande a lista de negative_doc_ids e coloca um registro por linha
# Obs.: isso vai colocar nan caso a lista esteja vazia. Será necessário filtrar depois
df = df.explode(classe_neg)
# Coloca no formato longo (por conta do df.explode, vai gerar um monte de duplicadas, que devem ser retiradas)
df = pd.melt(df, id_vars=['query', 'origin'], value_vars=[classe_neg, classe_pos])
df = df.drop_duplicates()
# Altera novamente o label das colunas variable e value:
df.rename(columns={'variable': 'label', 'value': 'passage'}, inplace=True)
# Filtra os nan
df = df[df.passage.notnull()]
# Mapeia a passagem (que só temos o doc_id) para o texto correto
df.passage = df.passage.map(lambda id: corpus_trec_covid[id])

# Embaralha o dataframe e faz o split dele em 2, um pra treino e um pra evaluation
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

#df = df[df.origin == 'leandro_carisio_01']

n_treino = int(perc_treino * len(df))
df_train = df.iloc[0:n_treino, :]
df_val = df.iloc[n_treino:, :]

df_train.head(10)

<ipython-input-8-7157e01c4076>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'variable': 'label', 'value': 'passage'}, inplace=True)


,query,origin,label,passage
0,Why has veterinary education failed to adapt t...,leonardo_avila_queries_v1,0,Suture length to wound length ratio for simple...
1,What other factors may influence the disaster ...,eduseiti_1000_queries_expansion_20230502_02,0,Knowledge and practices of isolation precautio...
2,How many large oligonucleotides were conserved...,eduseiti_1000_queries_expansion_20230502_02,1,Comparison of cytopathic and noncytopathic iso...
3,What is the impact of COVID-19 on the mental h...,pedrogengo_queries_inparsv1,1,Mental health among otolaryngology resident an...
4,What are the key priority areas and suggestion...,manoel_2k_generated_queries_20230501,0,Creating a Palliative Care Inpatient Response ...
5,Is there a risk of infectious disease transmis...,ricardo_primi_1k,0,Surgical Considerations for an Awake Tracheoto...
6,What histopathological changes occur in piglet...,eduseiti_1000_queries_expansion_20230502_02,0,Characterization of pneumonia due to Streptoco...
7,What are the differential diagnoses that need ...,marcospiau_1k_v1,0,Differential Ventilation Using Flow Control Va...
8,What are the key public health challenges invo...,marcospiau_1k_v1,0,Infectious diseases – A global challenge Abstr...
9,What is the difference between surgical and fi...,eduseiti_100_queries_expansion_20230501_01,1,What face mask for what use in the context of ...


In [9]:
print(df.label.value_counts())
print(df_train.label.value_counts())
print(df_val.label.value_counts())

1    23580
0    22585
Name: label, dtype: int64
1    21232
0    20316
Name: label, dtype: int64
1    2348
0    2269
Name: label, dtype: int64


In [10]:
pd.unique(df.origin)

array(['leonardo_avila_queries_v1',
       'eduseiti_1000_queries_expansion_20230502_02',
       'pedrogengo_queries_inparsv1',
       'manoel_2k_generated_queries_20230501', 'ricardo_primi_1k',
       'marcospiau_1k_v1', 'eduseiti_100_queries_expansion_20230501_01',
       'leandro_carisio_01', 'marcus_borela_1k_gptj6b_20230501',
       'thiago_laitz_1k_queries', 'mirelle_1k_generated_queries_20230501',
       'marcus_borela_1k_gptj6b_20230501_v2', 'thiago_vieira_1k_queries',
       'hugo_padovani_query_generation', 'gustavo_1k_cohere',
       'pedro_holanda_1k_generated_queries_20230502',
       'juliatessler_1000_queries',
       'thales_1k_generated_queries_20230429',
       'manoel_1k_generated_queries_20230430'], dtype=object)

In [11]:
import torch
from torch.utils import data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Definição para uma classe de Dataset
class Dataset(data.Dataset):
  # Recebe um dataframe do pandas. Precisa ter as colunas query, passage e label (0/1)
  def __init__(self, tokenizer, df, max_seq_length):
    self.max_seq_length = max_seq_length
    self.tokenizer = tokenizer

    # Já concatenas as query com as passagens e guarda em uma lista
    query_passage = df['query'] + ' [SEP] ' + df['passage']
    self.query_passage = query_passage.tolist()
    # Converte os labels para inteiros e guarda em uma lista
    self.labels = df.label.tolist()
    self.labels = [float(x) for x in self.labels]

    # Cria um cache vazio. Como tem treino em algumas épocas, guarda o encode no cache
    self.cache = {}

  def __len__(self):
    return len(self.query_passage)
  
  def get_token_type_ids(self, input_ids):
    idx_sep = input_ids.index(102)+1
    tam_seq = len(input_ids)
    token_type_ids = [0]*idx_sep + [1]*(tam_seq - idx_sep)

    # Apesar do tokenizer fazer isso, não precisa pois o attention_mask já zera.
    # for i in range(len(token_type_ids)):
    #   token_type_ids[i] = token_type_ids[i] if input_ids[i] != 0 else 0

    return token_type_ids

  def get_token_type_ids_from_slice(self, idx, matriz_input_ids):
    if isinstance(idx, slice):
      token_types = []
      for i in range(idx.start or 0, idx.stop or len(matriz_input_ids), idx.step or 1):
        token_types.append(self.get_token_type_ids(matriz_input_ids[i]))
      return token_types
    else:
      return self.get_token_type_ids(matriz_input_ids)

  def get_input_ids_e_labels(self, idx):
    input_ids_e_labels = self.tokenizer(self.query_passage[idx],
                                padding=True,
                                truncation=True,
                                max_length=self.max_seq_length)
    input_ids_e_labels['labels'] = self.labels[idx]

    input_ids_e_labels['token_type_ids'] = self.get_token_type_ids_from_slice(idx, input_ids_e_labels['input_ids'])

    return input_ids_e_labels

  def __getitem__(self, idx):
    # Guarda os itens tokenizados num dict e apenas recupera de lá, pra não ter que ficar tokenizando a cada época
    # Como estamos guardando no dict e idx é um slice, é necessário converter ele pra algo mapeável
    self.cache[str(idx)] = self.cache.get(str(idx), self.get_input_ids_e_labels(idx))
    return self.cache[str(idx)]

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BatchEncoding
from transformers import AdamW
from torch.utils.data import DataLoader

# Carrega tokenizador e modelo e cria um optimizer
tokenizer = AutoTokenizer.from_pretrained(nome_tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(nome_modelo).to(device)
optimizer = AdamW(model.parameters(), lr=lr)

# Cria datasets de treino e validação
dataset_train = Dataset(tokenizer, df_train, max_seq_length)
dataset_val = Dataset(tokenizer, df_val, max_seq_length)

# Cria dataloaders de treino e eval
collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Treino do modelo com AutoModelForSequenceClassification

In [13]:
%%time

from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
from statistics import mean

def evaluate(model, dataloader, set_name):
  losses = []
  correct = 0
  model.eval()
  with torch.no_grad():
    for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
      outputs = model(**batch.to(device))
      loss_val = outputs.loss
      losses.append(loss_val.cpu().item())
      # Só tem uma classe. Joga pra sigmoide e arredonda pro inteiro mais próximo
      preds = torch.round(torch.sigmoid(outputs.logits))
      correct += (preds.squeeze() == batch['labels']).sum().item()

  print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

def treina_modelo_com_automodel(model, optimizer, dataloader_train, dataloader_val, epoch_inicial, epochs):
  num_training_steps = epochs * len(dataloader_train)
  # Warm up is important to stabilize training.
  num_warmup_steps = int(num_training_steps * 0.1)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

  evaluate(model=model, dataloader=dataloader_val, set_name='Validation')

  # Training loop
  for epoch in tqdm(range(epoch_inicial, epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
      optimizer.zero_grad()
      # Observação:
      # Após checar os resultados da Mirelle, aprendi que no treinamento temos que desconsiderar
      # o token_type_ids (mas esse token_type_ids tem que ser usado na inferência!) #outputs = model(**batch.to(device))
      b = batch.to(device)
      outputs = model(input_ids=b['input_ids'], attention_mask=b['attention_mask'], labels=b['labels'])
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      scheduler.step()
      train_losses.append(loss.cpu().item())

    print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
    model.save_pretrained(f'{url_salvar_modelos_intermediarios}{epoch+1}/')
    evaluate(model=model, dataloader=dataloader_val, set_name='Validation')
    print('---------------------------------------------------------------------')

if treinar_modelo:
  treina_modelo_com_automodel(model, optimizer, dataloader_train, dataloader_val, 0, epochs=5)

Validation:   0%|          | 0/145 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Validation loss: 40.127; Validation accuracy: 0.885


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train:   0%|          | 0/1299 [00:00<?, ?it/s]

...

Validation loss: 0.562; Validation accuracy: 0.867
---------------------------------------------------------------------


Train:   0%|          | 0/1299 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.03


Validation:   0%|          | 0/145 [00:00<?, ?it/s]

Validation loss: 0.392; Validation accuracy: 0.864
---------------------------------------------------------------------
CPU times: user 27min 18s, sys: 5.95 s, total: 27min 24s
Wall time: 27min 18s


## BM25

Implementação das classes de índice invertido e BM25.

In [14]:
%%time
from pyserini.analysis import Analyzer, get_lucene_analyzer
from collections import Counter
import array
import pickle
import math

# Definição de uma classe para índice invertido
class IndiceInvertido:
  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0
    # Tokenizador
    self.lucene_analyser = Analyzer(get_lucene_analyzer(stemmer='porter'))

  def tokenizar(self, texto):
    return self.lucene_analyser.analyze(texto)

  def adiciona_doc(self, id_doc, conteudo_doc=None):
    tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)
    
    self.n_docs += 1
    self.tamanho_doc[id_doc] = len(tokens)

class BM25:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.bias_adicionar_ao_idf = bias_adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
    self.reinicia_score_dos_indices()

  def reinicia_score_dos_indices(self):
    for token in self.indiceInvertido.indice.keys():
      self.indiceInvertido.indice[token].pop('score', None)
  
  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_score_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    idf = self.indiceInvertido.indice[token]['idf']
    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])   
    bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indiceInvertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['score'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']
      
      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if 'score' not in self.indiceInvertido.indice[token].keys():
        self.calcula_score_para_um_token_e_salva(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token]['score']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 496 ms, sys: 41.2 ms, total: 537 ms
Wall time: 706 ms


In [15]:
iidx_trec_covid = IndiceInvertido()

# Carrega todo o corpus no índice invertido
cnt = 0
for id, titulo_texto in corpus_trec_covid.items():
  iidx_trec_covid.adiciona_doc(id, titulo_texto)

  cnt += 1
  if cnt % 10000 == 0:
    print(f'{cnt} docs indexados')

buscador = BM25(iidx_trec_covid, 0.82, 0.68, 1)

10000 docs indexados
20000 docs indexados
30000 docs indexados
40000 docs indexados
50000 docs indexados
60000 docs indexados
70000 docs indexados
80000 docs indexados
90000 docs indexados
100000 docs indexados
110000 docs indexados
120000 docs indexados
130000 docs indexados
140000 docs indexados
150000 docs indexados
160000 docs indexados
170000 docs indexados


## Reranking com o modelo carregado via AutoModelForSequenceClassification

In [16]:
%%time
from pyserini.search import get_qrels

def reranking_docs(model, query, doc_ids):
  model.eval()
  
  # Cria um dataset para fazer o reranking
  passagens = [ corpus_trec_covid[id] for id in doc_ids ]
  queries = [query]*len(doc_ids)
  labels = [1]*len(doc_ids) # é eval, tanto faz o valor. Ele não será utilizado, mas precisamos cadastrar aqui
  df_reranking = pd.DataFrame({'query': queries, 'passage': passagens, 'label': labels})

  dataset_reranking = Dataset(tokenizer, df_reranking, max_seq_length)
  dataloader_reranking = DataLoader(dataset_reranking, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

  score_todos_docs = []
  with torch.no_grad():
    for batch in tqdm(dataloader_reranking, mininterval=0.5, desc=f'Query {query}', disable=False):
      outputs = model(**batch.to(device))
      #score = torch.sigmoid(outputs.logits).squeeze().tolist()
      score = outputs.logits.squeeze().tolist()
      score_todos_docs.extend( score )

  docs_com_score = list(zip(doc_ids, score_todos_docs))

  torch.cuda.empty_cache()

  return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

# Roda todas as queries
def run_all_queries_bm25_e_rerank(file, model, buscador, hits=1000):
  with open(file, 'w') as runfile:
    cnt = 0
    print('Running {} queries in total'.format(len(queries_trec_covid)))
    for query in queries_trec_covid:
      id = query['id']
      texto_query = query['texto']
      
      if cnt % 10 == 0:
        print(f'{cnt} queries completed')
      cnt += 1

      # Usa o BM25 para buscar
      docs_scores = buscador.pesquisar(texto_query)
      # Pega só os 1000 primeiros pra fazer o reranking
      n_reranking = min(hits, len(docs_scores))
      docs_scores = docs_scores[0:n_reranking]
      # Agora faz o reranking
      docs, scores = zip(*docs_scores)
      docs_score = reranking_docs(model, texto_query, docs)
      for i in range(0, len(docs_score)): # Pega os primeiros 1000 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} BM_25_RERANKING_MINILM\n'.format(id, docs_score[i][0], i+1, docs_score[i][1]))



CPU times: user 2.35 s, sys: 130 ms, total: 2.48 s
Wall time: 1.87 s


Sem fine-tuning

In [ ]:
%%time

# Testa com o modelo sem fine-tuning
model_eval_original = AutoModelForSequenceClassification.from_pretrained(nome_modelo).to(device)

print('Rodando pesquisa com BM25 e reranking usando miniLM')
run_all_queries_bm25_e_rerank('run-inpars-msmarco.txt', model_eval_original, buscador)
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-inpars-msmarco.txt #type: ignore

Rodando pesquisa com BM25 e reranking usando miniLM
Running 50 queries in total
0 queries completed


Query what is the origin of COVID-19:   0%|          | 0/63 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Query how does the coronavirus respond to changes in the weather:   0%|          | 0/63 [00:00<?, ?it/s]

Query will SARS-CoV2 infected people develop immunity? Is cross protection possible?:   0%|          | 0/63 [0…

...

Query what are the health outcomes for children who contract COVID-19?:   0%|          | 0/63 [00:00<?, ?it/s]

Query what are the benefits and risks of re-opening schools in the midst of the COVID-19 pandemic?:   0%|     …

Query do individuals who recover from COVID-19 show sufficient immune response, including antibody levels and …

Query what is known about an mRNA vaccine for the SARS-CoV-2 virus?:   0%|          | 0/63 [00:00<?, ?it/s]

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 6.31MB/s]                
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-inpars-msmarco.txt']
Results:
ndcg_cut_10           	all	0.7131
CPU times: user 5min 56s, sys: 2.78 s, total: 5min 58s
Wall time: 6min 5s


Com fine-tuning (reranking de 100 resultados):

In [19]:
%%time
model_eval_fine_tuning = AutoModelForSequenceClassification.from_pretrained(modelo_eval).to(device)

print('Rodando pesquisa com BM25 e reranking usando miniLM')
run_all_queries_bm25_e_rerank('run-inpars.txt', model_eval_fine_tuning, buscador, hits=100)
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-inpars.txt #type: ignore

Rodando pesquisa com BM25 e reranking usando miniLM
Running 50 queries in total
0 queries completed


Query what is the origin of COVID-19:   0%|          | 0/4 [00:00<?, ?it/s]

Query how does the coronavirus respond to changes in the weather:   0%|          | 0/4 [00:00<?, ?it/s]

Query will SARS-CoV2 infected people develop immunity? Is cross protection possible?:   0%|          | 0/4 [00…

Query what causes death from Covid-19?:   0%|          | 0/4 [00:00<?, ?it/s]

...

Query what are the health outcomes for children who contract COVID-19?:   0%|          | 0/4 [00:00<?, ?it/s]

Query what are the benefits and risks of re-opening schools in the midst of the COVID-19 pandemic?:   0%|     …

Query do individuals who recover from COVID-19 show sufficient immune response, including antibody levels and …

Query what is known about an mRNA vaccine for the SARS-CoV-2 virus?:   0%|          | 0/4 [00:00<?, ?it/s]

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-inpars.txt']
Results:
ndcg_cut_10           	all	0.7470
CPU times: user 26.4 s, sys: 509 ms, total: 26.9 s
Wall time: 33.7 s


Com fine-tuning (reranking de 1000 resultados):

In [20]:
%%time
model_eval_fine_tuning = AutoModelForSequenceClassification.from_pretrained(modelo_eval).to(device)

print('Rodando pesquisa com BM25 e reranking usando miniLM')
run_all_queries_bm25_e_rerank('run-inpars.txt', model_eval_fine_tuning, buscador, hits=1000)
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-inpars.txt #type: ignore

Rodando pesquisa com BM25 e reranking usando miniLM
Running 50 queries in total
0 queries completed


Query what is the origin of COVID-19:   0%|          | 0/32 [00:00<?, ?it/s]

Query how does the coronavirus respond to changes in the weather:   0%|          | 0/32 [00:00<?, ?it/s]

Query will SARS-CoV2 infected people develop immunity? Is cross protection possible?:   0%|          | 0/32 [0…

Query what causes death from Covid-19?:   0%|          | 0/32 [00:00<?, ?it/s]

...

Query what are the health outcomes for children who contract COVID-19?:   0%|          | 0/32 [00:00<?, ?it/s]

Query what are the benefits and risks of re-opening schools in the midst of the COVID-19 pandemic?:   0%|     …

Query do individuals who recover from COVID-19 show sufficient immune response, including antibody levels and …

Query what is known about an mRNA vaccine for the SARS-CoV-2 virus?:   0%|          | 0/32 [00:00<?, ?it/s]

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-inpars.txt']
Results:
ndcg_cut_10           	all	0.7518
CPU times: user 2min 52s, sys: 911 ms, total: 2min 53s
Wall time: 2min 57s
